In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer
from emoji import demojize
from nltk.tokenize import TweetTokenizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, multilabel_confusion_matrix

DATASET_DIR = "../dataset/ptc_adjust/"

2023-11-09 14:56:44.363027: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-09 14:56:48.653318: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-09 14:56:48.657249: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-11-09 14:56:48.657261: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
train = pd.read_csv(DATASET_DIR+"ptc_preproc_train.csv", sep=";").dropna(subset=["text", "label"])[["text", "label"]]
train = train.drop_duplicates(subset=["text"])
test = pd.read_csv(DATASET_DIR+"ptc_preproc_test.csv", sep=";").dropna(subset=["text", "label"])[["text", "label"]]
test = test.drop_duplicates(subset=["text"])

# This pd.DataFrame should be empty
pd.concat([test[test.text.isnull()],train[train.text.isnull()]])

,text,label


In [3]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, normalization=True)
model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
data = pd.concat((train["text"], test["text"]))
tokens = data.apply((lambda x: tokenizer.encode(
    x,
    # add_special_tokens=True,
    truncation=True,
    # padding=True,
    max_length=128
)))
tokens

0               [0, 42284, 3726, 47691, 111, 21629, 5, 2]
1       [0, 1401, 158, 4545, 19, 756, 8382, 124901, 22...
2                    [0, 262, 88981, 3291, 8408, 1135, 2]
3       [0, 70, 64857, 450, 26038, 70, 8999, 70, 14069...
4       [0, 581, 17274, 5608, 8306, 24209, 10, 36541, ...
                              ...                        
1375    [0, 10, 6782, 23972, 62, 25965, 674, 91375, 53...
1376                   [0, 5879, 9, 2943, 214, 131161, 2]
1377                           [0, 17202, 7941, 16070, 2]
1378                [0, 51, 14473, 5874, 38526, 83024, 2]
1379                                 [0, 58867, 46667, 2]
Name: text, Length: 5674, dtype: object

In [5]:
padded = np.array([token+[0]*(128-len(token)) for token in tokens.values])
np.array(padded).shape

(5674, 128)

In [6]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(5674, 128)

In [7]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

# extract [CLS] token hidden representation from output layer
features = last_hidden_states[0][:,0,:].numpy()
print(f'features shape: {features.shape}')


features shape: (5674, 768)


In [8]:
train_features, test_features = features[:-len(test)], features[len(train):]
print(f'train features: {len(train_features)}')
print(f'test features: {len(test_features)}')

train features: 4464
test features: 1210


In [56]:
train_labels, test_labels = train["label"].to_numpy(), test["label"].to_numpy()

train_labels = [label.split(",") for label in train_labels]
test_labels = [label.split(",") for label in test_labels]

[(idx,labels) for idx,labels in enumerate(test_labels) if len(labels)>1]

[(50, ['Loaded_Language', 'Slogans']),
 (110,
  ['Appeal_to_Authority',
   'Appeal_to_fear-prejudice',
   'Exaggeration_Minimisation',
   'Repetition']),
 (111, ['Appeal_to_Authority', 'Exaggeration_Minimisation']),
 (114, ['Exaggeration_Minimisation', 'Repetition']),
 (115, ['Appeal_to_fear-prejudice', 'Repetition']),
 (116, ['Appeal_to_Authority', 'Appeal_to_fear-prejudice']),
 (122, ['Appeal_to_Authority', 'Repetition']),
 (125, ['Appeal_to_Authority', 'Appeal_to_fear-prejudice']),
 (139, ['Appeal_to_Authority', 'Exaggeration_Minimisation']),
 (181, ['Appeal_to_Authority', 'Exaggeration_Minimisation']),
 (191, ['Causal_Oversimplification', 'Exaggeration_Minimisation']),
 (210, ['Appeal_to_Authority', 'Exaggeration_Minimisation']),
 (252, ['Appeal_to_Authority', 'Repetition']),
 (253, ['Appeal_to_Authority', 'Repetition']),
 (255, ['Appeal_to_Authority', 'Exaggeration_Minimisation']),
 (267, ['Appeal_to_fear-prejudice', 'Black-and-White_Fallacy']),
 (282, ['Appeal_to_fear-prejudice',

In [44]:
# train_labels, test_labels = train["label"].to_numpy(), test["label"].to_numpy()

mlb = MultiLabelBinarizer()
train_labels_binarized = mlb.fit_transform(train_labels)
test_labels_binarized = mlb.fit_transform(test_labels)


In [45]:

ff = MLPClassifier(
    random_state=1,
    max_iter=400,
    alpha=0.001,
    shuffle=True,
    early_stopping=True,
    verbose=True
).fit(train_features, train_labels_binarized)

Iteration 1, loss = 6.42800591
Validation score: 0.000000
Iteration 2, loss = 3.18817855
Validation score: 0.069351
Iteration 3, loss = 3.01619315
Validation score: 0.098434
Iteration 4, loss = 2.93351221
Validation score: 0.167785
Iteration 5, loss = 2.86609473
Validation score: 0.131991
Iteration 6, loss = 2.80074188
Validation score: 0.172260
Iteration 7, loss = 2.75014343
Validation score: 0.190157
Iteration 8, loss = 2.70152061
Validation score: 0.154362
Iteration 9, loss = 2.66371236
Validation score: 0.158837
Iteration 10, loss = 2.62773229
Validation score: 0.185682
Iteration 11, loss = 2.59538986
Validation score: 0.230425
Iteration 12, loss = 2.57292016
Validation score: 0.208054
Iteration 13, loss = 2.55264689
Validation score: 0.219239
Iteration 14, loss = 2.52619128
Validation score: 0.201342
Iteration 15, loss = 2.50599701
Validation score: 0.210291
Iteration 16, loss = 2.47995805
Validation score: 0.230425
Iteration 17, loss = 2.46700481
Validation score: 0.223714
Iterat

In [46]:
test_predicted_labels_binarized = ff.predict(test_features)
micro_f1 = f1_score(test_labels_binarized, test_predicted_labels_binarized, average="micro")
acc = accuracy_score(test_labels_binarized, test_predicted_labels_binarized)
prec = precision_score(test_labels_binarized, test_predicted_labels_binarized, average="micro")
rec = recall_score(test_labels_binarized, test_predicted_labels_binarized, average="micro")
print(f'micro-f1: {micro_f1}')
print(f'accuracy: {acc}')
print(f'micro-precision: {prec}')
print(f'micro-recall: {rec}')

micro-f1: 0.383618455158113
accuracy: 0.2834710743801653
micro-precision: 0.5522388059701493
micro-recall: 0.29388403494837173


In [47]:
cf_mtx = multilabel_confusion_matrix(test_labels_binarized, test_predicted_labels_binarized)
cf_mtx

array([[[1161,    0],
        [  49,    0]],

       [[1082,    0],
        [ 127,    1]],

       [[1206,    0],
        [   4,    0]],

       [[1185,    0],
        [  25,    0]],

       [[1179,    0],
        [  31,    0]],

       [[1134,    5],
        [  67,    4]],

       [[1111,   14],
        [  82,    3]],

       [[1131,    1],
        [  67,   11]],

       [[ 565,  250],
        [  94,  301]],

       [[1006,   25],
        [ 130,   49]],

       [[1204,    0],
        [   6,    0]],

       [[1198,    0],
        [  12,    0]],

       [[1198,    0],
        [  12,    0]],

       [[1096,    1],
        [ 112,    1]],

       [[1175,    1],
        [  34,    0]],

       [[1208,    0],
        [   2,    0]],

       [[1191,    3],
        [  16,    0]],

       [[1191,    0],
        [  19,    0]]])

In [63]:
test_labels_array = []
for label_list in test_labels:
    for label in label_list:
        test_labels_array.append(label)

(np.unique(np.array(test_labels_array)))

array(['Appeal_to_Authority', 'Appeal_to_fear-prejudice', 'Bandwagon',
       'Black-and-White_Fallacy', 'Causal_Oversimplification', 'Doubt',
       'Exaggeration_Minimisation', 'Flag-Waving', 'Loaded_Language',
       'Name_Calling_Labeling',
       'Obfuscation_Intentional_Vagueness_Confusion', 'Red_Herring',
       'Reductio_ad_hitlerum', 'Repetition', 'Slogans', 'Straw_Men',
       'Thought-terminating_Cliches', 'Whataboutism'], dtype='<U43')